In [7]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import classification_report, confusion_matrix


In [89]:
numerical_columns = ["SLHC","HST","HCT","MCV","MCH","MCHC","RDWCV","SLTC","SLBC"]
numerical_columns2 = ["FERRITIN","FE"]    
model1_name = '../models/model_9_features.pkl'
model2_name = '../models/model_2_features.pkl'


numerical_columns = ["SLHC","HST","HCT","MCV","MCH","MCHC","RDWCV","SLTC","SLBC"]
numerical_columns2 = ["FERRITIN","FE", "HBA1","HBA2"]  
model1_name = '../models/model_9_features.pkl'
model2_name = '../models/model_4_features.pkl'

numerical_columns = ["SLHC","HST","HCT","MCV","MCH","MCHC","RDWCV","SLTC","SLBC","FERRITIN","FE"]
numerical_columns2 = ["HBA1","HBA2"]   
model1_name = '../models/model_11_features.pkl'
model2_name = '../models/model_hba.pkl'

In [90]:

def read_main_data(file_path = '../datasets/splited_train_test.csv'):
    global numerical_columns

    df = pd.read_csv(file_path, low_memory=False)
    df.drop_duplicates(subset=['PID'], keep='last', inplace=True)
#     df = df[~(df.sheet.isin(['d1']) & (df.thalas == 0))]
    
    num_df = df[numerical_columns].apply(pd.to_numeric, errors='coerce', downcast='float')
    
    df = df.reindex(num_df.dropna().index)
    df[numerical_columns] = num_df.dropna()

    test_df = df[df.train==0]
    train_df = df[df.train==1]

    return train_df, test_df






def load_model(model_name):
    with open(model_name, 'rb') as f:
        return pickle.load(f)

model1 = load_model(model1_name)
model2 = load_model(model2_name)

In [91]:

def evaluate(model1, model2, df, desc=''):
    threshold = 0.6
    X = df[numerical_columns]
    y = df['thalas']
    preds1 = np.array(model1.predict(X))
    preds1 = np.where(preds1 >= threshold, 1, 0) # dự đoán model 1
    print('---------------------------------------------------')
    print(f"evaluation results {desc}:\n")
    print(classification_report(y, preds1))
    print('---------------------------------------------------')
    # mask = preds != y
    mask = (preds1 == 0) & ((X.MCV < 85) | (X.MCH < 28))
    X2 = df[numerical_columns2][mask] # lấy những sample khả năng bị dự đoán sai ra để đưa vào mô hình 2
    print(mask.sum())
    preds2 = np.array(model2.predict(X2))
    preds2 = np.where(preds2 >= threshold, 1, 0)
    y1 = y[~mask]
    y2 = y[mask]
    y3 = np.concatenate([y1, y2])
    print(classification_report(y3, np.concatenate([preds1[~mask], preds2])))

_, test_df = read_main_data()
evaluate(model1, model2, test_df)

---------------------------------------------------
evaluation results :

              precision    recall  f1-score   support

         0.0       0.92      0.79      0.85      1000
         1.0       0.82      0.93      0.87      1000

    accuracy                           0.86      2000
   macro avg       0.87      0.86      0.86      2000
weighted avg       0.87      0.86      0.86      2000

---------------------------------------------------
226
              precision    recall  f1-score   support

         0.0       0.99      0.66      0.79      1000
         1.0       0.74      0.99      0.85      1000

    accuracy                           0.82      2000
   macro avg       0.87      0.82      0.82      2000
weighted avg       0.87      0.82      0.82      2000



In [85]:
test_df.MCV.describe()

count    2000.000000
mean       78.788414
std        10.956099
min        44.099998
25%        69.475000
50%        79.900002
75%        88.300003
max       113.099998
Name: MCV, dtype: float64